In [133]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [285]:
import os.path
import subprocess
import subprocess
import pandas as pd
import time
from bs4 import BeautifulSoup
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import zipfile
from six.moves import urllib
from pdb2sql import pdb2sql
import numpy as np
import glob
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import NoSuchElementException

In [286]:
fname = "testseq"
fin = []
for record in SeqIO.parse(fname+".fasta", "fasta"):
    rec_id = (">" + record.id)
    seq = record.seq
    PATH = "/home/dell/WebDrivers/chromedriver"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless')
    chrome_options.add_argument('window-size=1920x1080')
    chrome_options.add_argument("disable-gpu")
    s = Service(PATH)
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.implicitly_wait(75)
    driver.get("http://pfam.xfam.org/")
    rec_search = driver.find_element(by=By.XPATH, value='/html/body/div[5]/div[2]/ul/li[5]').click()
    recordid = record.id[3:9]
    searchbox = driver.find_element(by=By.XPATH, value='/html/body/div[5]/div[7]/form/div[1]/input[1]').send_keys(recordid)
    driver.find_element(by=By.XPATH, value='/html/body/div[5]/div[7]/form/div[2]/a[1]').click()
    timeout = 175
    try:
        element_present = EC.presence_of_element_located((By.ID, 'content'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        break
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    div = soup.find_all('table', {'id': 'imageKey'})
    table = pd.read_html(str(div))
    for t in table:
        t['id'] = rec_id
    fin.append(table)
    driver.close()
    print("Finished getting PFAM Results Table data")

Finished getting PFAM Results Table data
Finished getting PFAM Results Table data


In [298]:
df3 = pd.DataFrame()
for f in fin:
    df = f[0]
    df2 = df.T.reset_index(level=1).T
    df2 = df2.drop('level_1', axis=0)
    df3 = pd.concat([df3, df2])
data = []
for record in SeqIO.parse(fname + ".fasta", "fasta"):
    data.append([">" + record.id, "".join(record.seq)])
df_seq = pd.DataFrame(data, columns=['id', 'seq'])
df3=df3.merge(df_seq, on='id')
df3['domain_seq'] = df3.apply(lambda x: x['seq'][x['Start'] - 1:x['End']], axis=1)
df3['new_id'] = df3.apply(lambda x: ">"+x['id'][4:10] + "_" + str(x['Start']) + "-" + str(x['End']), axis=1)
df3

,Source,Domain,Start,End,id,seq,domain_seq,new_id
0,Pfam,MMR_HSR1,4,119,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,VVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANK,>P0A6P5_4-119
1,low_complexity,NaN,55,69,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,IDTGGIDGTEDGVET,>P0A6P5_55-69
2,disorder,NaN,64,66,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,EDG,>P0A6P5_64-66
3,disorder,NaN,116,118,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,VAN,>P0A6P5_116-118
4,disorder,NaN,177,196,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,EYWAQFEAEENGEEEEEDDF,>P0A6P5_177-196
5,low_complexity,NaN,183,193,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,EAEENGEEEEE,>P0A6P5_183-193
6,Pfam,MMR_HSR1,204,322,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHG

In [300]:
ln = len(df3['Domain'])
df4 = pd.DataFrame()
for x in range(ln):
    if type(df3['Domain'].iloc[x]) is str:
        if df3['Domain'].iloc[x].find('MMR_HSR1') == 0:
            df4 = pd.concat([df4, df3.iloc[x:x + 1, :]])
    else:
        continue;
df4.set_index('new_id', inplace=False)
df4

,Source,Domain,Start,End,id,seq,domain_seq,new_id
0,Pfam,MMR_HSR1,4,119,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,VVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANK,>P0A6P5_4-119
6,Pfam,MMR_HSR1,204,322,>sp|P0A6P5|DER_ECOLI,MVPVVALVGRPNVGKSTLFNRLTRTRDALVADFPGLTRDRKYGRAEIEGREFICIDTGGIDGTEDGVETRMAEQSLLAIEEADVVLFMVDARAGLMPADEAIAKHLRSREKPTFLVANKTDGLDPDQAVVDFYSLGLGEIYPIAASHGRGVLSLLEHVLLPWMEDLAPQEEVDEDAEYWAQFEAEENGEEEEEDDFDPQSLPIKLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNKWDGLSQEVKEQVKETLDFRLGFIDFARVHFISALHGSGVGNLFESVREAYDSSTRRVGTSMLTRIMTMAVEDHQPPLVRGRRVKLKYAHAGGYNPPIVVIHGNQVKDLPDSYKRYLMNYFRKSLDVMGSPIRIQFKEGENPYANKRNTLTPTQMRKRKRLMKHIKKNK,KLAIVGRPNVGKSTLTNRILGEERVVVYDMPGTTRDSIYIPMERDGREYVLIDTAGVRKRGKITDAVEKFSVIKTLQAIEDANVVMLVIDAREGISDQDLSLLGFILNSGRSLVIVVNK,>P0A6P5_204-322
9,Pfam,MMR_HSR1,5,120,>sp|P50743|DER_BACSU,MGKPVVAIVGRPNVGKSTIFNRIAGERISIVEDTPGVTRDRIYSSAEWLNYDFNLIDTGGIDIGDEPFLAQIRQQAEIAMDEADVIIFMVNGREGVTAADEEVAKILYRTKKPVVLAVNKLDNTEMRANIYDFYSLGFGEPYPISGTHGLGLGDLLDAVAEHFKNIPETKYNEEVIQFCLIGRPNVGKSSLVNAMLGEERVIVSNVAGTTRDAVDTSFTYNQQEFVIVDTAGMRKKGKVYETTEKYSVLRALKAIDRSEVVAVVLDGEEGIIEQDKRIAGYAHEAGKAVVIVVNKWDAVDKDESTMKEFEENIRDHFQFLDYAPILFMSALTKKRIHTLMPAIIKASENHSLRVQTNVLNDVIMDAVAMNPTPTHNGSRLKIYYATQVSVKPPSFVVFVNDPELMHFSYERFLENRIRDAFGFEGTPIKIFARARK,VVAIVGRPNVGKSTIFNRIAGERISIVEDTPGVTRDRIYSSAEWLNYDFNLIDTGGIDIGDEPFLAQIRQQAEIAMDEADVIIFMVNGREGVTAADEEVAKILYRTKKPVVLAVNK,>P50743_5-120
11,Pfam,MMR_HSR1,177,295,>sp|P50743|DER_BACSU,MGKPVVAIVGRPNVGKSTIFNRIAGERISIVEDTPGVTRDRIYSSAEWLNYDFNLIDTGGIDIGDEPFLAQIRQQAEIAMDEADVIIFMVNGREGVTAADEEVAKILYRTKKPVVLAVNKLDNTEMRANIYDFYSLGFGEPYPISGTHGLGLGDLLDAVAEHFKNIPETKYNEEVIQFCLIGRPNVGKSSLVNAMLGEERVIVSNVAGTTRDAVDTSFTYNQQEFVIVDTAGMRKKGKVYETTEKYSVLRALKAIDRSEVVAVVLDGEEGIIEQDKRIAGYAHEAGKAVVIVVNKWDAVDKDESTMKEFEENIRDHFQFLDYAPILFMSALTKKRIHTLMPAIIKASENHSLRVQTNVLNDVIMDAVAMNPTPTHNGSRLKIYYATQVSVKPPSFVVFVNDPELMHFSYERFLENRIRDAFGFEGTPIKIFARARK,QFCLIGRPNVGKSSLVNAMLGEERVIVSNVAGTTRDAVDTSFTYNQQEFVIVDTAGMRKKGKVYETTEKYSVLRALKAIDRSEVVAVVLDGEEGIIEQDKRIAGYAHEAGKAVVIVVNK,>P50743_177-295


In [301]:
with open("domain_seq.fasta", "w") as file:        
    for idx, row in df4.iterrows():                
        file.write(row['new_id'] + "\n")           
        file.write(row['domain_seq'] + "\n")
print("wrote sliced sequences in domain_seq.fasta")

wrote sliced sequences in domain_seq.fasta


In [302]:
align = "./probcons/probcons -clustalw domain_seq.fasta > domain_seq_aligned.aln" 
subprocess.call(align, shell=True)                                                
print ("Aligned domain sequences.")                                               

Aligned domain sequences.



PROBCONS version 1.12 - align multiple protein sequences and print to standard output
Written by Chuong Do

Using parameter set:
    initDistrib[] = { 0.6814756989 8.615339902e-05 8.615339902e-05 0.1591759622 0.1591759622 }
        gapOpen[] = { 0.0119511066 0.0119511066 0.008008334786 0.008008334786 }
      gapExtend[] = { 0.3965826333 0.3965826333 0.8988758326 0.8988758326 }

Loading sequence file: domain_seq.fasta
Alignment tree: ((P0A6P5_4-119 P50743_5-120) (P0A6P5_204-322 P50743_177-295))




In [200]:
os.getcwd()

'/home/dell/Desktop/CPDAS/venv'

In [303]:
with open('domain_seq_aligned.aln', 'r') as file:
    data = file.readlines()
data[0] = "CLUSTAL W (1.82) multiple sequence alignment\n"
with open('domain_seq_aligned.aln', 'w') as file:
    file.writelines(data)

In [304]:
cons_score = "java -jar compbio-conservation-1.1.jar -i=domain_seq_aligned.aln -o=cons_out.csv"     
norm_score = "java -jar compbio-conservation-1.1.jar -i=domain_seq_aligned.aln -n -o=norm_out.csv"  
subprocess.call(norm_score, shell=True)                                                             
subprocess.call(cons_score, shell=True)                                                             
print("calculated conservation scores for the alignment")                                           

calculated conservation scores for the alignment


In [305]:
df5 = pd.read_csv('cons_out.csv', delimiter='\s+', header=None)
df5.set_index([0], inplace=True)                               
df5.reset_index                                                
df5.index.rename('Method', inplace=True)                       
df5.columns = pd.RangeIndex(0, len(df5.columns))               
df5.to_csv('conservation_scores.csv')     
df5

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118
Method,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#KABAT,6.0000,6.0000,2.6667,4.0000,2.6667,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.6667,2.6667,6.0000,1.0000,2.6667,6.0000,6.0000,2.6667,2.6667,4.0000,6.0000,6.0000,6.0000,1.0000,16.0000,2.6667,16.0000,2.6667,1.0000,6.0000,1.0000,1.0000,1.0000,6.0000,6.0000,2.6667,16.0000,6.0000,6.0000,2.6667,16.0000,16.0000,6.0000,6.0000,2.6667,2.6667,6.0000,6.0000,2.6667,1.0000,1.0000,4.0000,1.0000,6.0000,4.0000,6.0000,16.0000,6.0000,2.0000,8.0000,8.0000,4.0000,16.0000,6.0000,2.6667,6.0000,16.0000,6.0000,6.0000,16.0000,6.0000,6.0000,2.6667,16.0000,1.0000,2.6667,4.0000,6.0000,2.6667,6.0000,1.0000,2.6667,16.0000,6.0000,4.0000,6.0000,2.6667,4.0000,2.6667,2.6667,1.0000,6.0000,16.0000,16.0000,4.0000,1.0000,6.0000,16.0000,6.0000,2.6667,4.0000,16.0000,6.0000,16.0000,16.0000,16.0000,6.0000,2.6667,6.0000,6.0000,2.6667,4.0000,2.6667,2.6667,1.0000,1.0000
#JORES,12.0000,12.0000,4.0000,4.5000,4.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,4.0000,4.0000,12.0000,1.0000,4.0000,12.0000,12.0000,4.0000,4.0000,4.5000,12.0000,12.0000,12.0000,1.0000,36.0000,4.0000,36.0000,4.0000,1.0000,12.0000,1.0000,1.0000,1.0000,12.0000,12.0000,4.0000,36.0000,12.0000,12.0000,4.0000,36.0000,36.0000,12.0000,12.0000,4.0000,4.0000,12.0000,12.0000,4.0000,1.0000,1.0000,4.5000,1.0000,12.0000,4.5000,12.0000,36.0000,12.0000,6.0000,6.0000,6.0000,4.5000,36.0000,12.0000,4.0000,12.0000,36.0000,12.0000,12.0000,36.0000,12.0000,12.0000,4.0000,36.0000,1.0000,4.0000,4.5000,12.0000,4.0000,12.0000,1.0000,4.0000,36.0000,12.0000,4.5000,12.0000,4.0000,4.5000,4.0000,4.0000,1.0000,12.0000,36.0000,36.0000,4.5000,1.0000,12.0000,36.0000,12.0000,4.0000,4.5000,36.0000,12.0000,36.0000,36.0000,36.0000,12.0000,4.0000,12.0000,12.0000,4.0000,4.5000,4.0000,4.0000,1.0000,1.0000
#SCHNEIDER,0.3471,0.3471,0.1877,0.2314,0.1877,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1877,0.1877,0.3471,0.0000,0.1877,0.3471,0.3471,0.1877,0.1877,0.2314,0.3471,0.3471,0.3471,0.0000,0.4628,0.1877,0.4628,0.1877,0.0000,0.3471,0.0000,0.0000,0.0000,0.3471,0.3471,0.1877,0.4628,0.3471,0.3471,0.1877,0.4628,0.4628,0.3471,0.3471,0.1877,0.1877,0.3471,0.3471,0.1877,0.0000,0.0000,0.2314,0.0000,0.3471,0.2314,0.3471,0.4628,0.3471,0.1157,0.2314,0.2314,0.2314,0.4628,0.3471,0.1877,0.3471,0.4628,0.3471,0.3471,0.4628,0.3471,0.3471,0.1877,0.4628,0.0000,0.1877,0.2314,0.3471,0.1877,0.3471,0.0000,0.1877,0.4628,0.3471,0.2314,0.3471,0.1877,0.2314,0.1877,0.1877,0.0000,0.3471,0.4628,0.4628,0.2314,0.0000,0.3471,0.4628,0.3471,0.1877,0.2314,0.4628,0.3471,0.4628,0.4628,0.4628,0.3471,0.1877,0.3471,0.3471,0.1877,0.2314,0.1877,0.1877,0.0000,0.0000
#SHENKIN,16.9706,16.9706,10.5286,12.0000,10.5286,6.0000,6.0000,6.0000,6.0000,6.0000,6.0000,6.0000,6.0000,10.5286,10.5286,16.9706,6.0000,10.5286,16.9706,16.9706,10.5286,10.5286,12.0000,16.9706,16.9706,16.9706,6.0000,24.0000,10.5286,24.0000,10.5286,6.0000,16.9706,6.0000,6.0000,6.0000,16.9706,16.9706,10.5286,24.0000,16.9706,16.9706,10.5286,24.0000,24.0000,16.9706,16.9706,10.5286,10.5286,16.9706,16.9706,10.5286,6.0000,6.0000,12.0000,6.0000,16.9706,12.0000,16.9706,24.0000,16.9706,8.4853,12.0000,12.0000,12.0000,24.0000,16.9706,10.5286,16.9706,24.0000,16.9706,16.9706,24.0000,16.9706,16.9706,10.5286,24.0000,6.0000,10.5286,12.0000,16.9706,10.5286,16.9706,6.0000,10.5286,24.0000,16.9706,12.0000,16.9706,10.5286,12.0000,10.5286,10.5286,6.0000,16.9706,24.0000,24.0000,12.0000,6.0000,16.9706,24.0000,16.9706,10.5286,12.0000,24.0000,16.9706,24.0000,24.0000,24.0000,16.9706,10.5286,16.9706,16.9706,10.5286,12.0000,10.

In [306]:
df6 = pd.read_csv('norm_out.csv', delimiter='\s+', header=None)
df6.set_index([0], inplace=True)                               
df6.reset_index                                                
df6.index.rename('Method', inplace=True)                       
df6.columns = pd.RangeIndex(0, len(df5.columns))              
df6.to_csv('normalized_scores.csv')                            
NormScoreDict = df6.to_dict(orient='index')                    
df6

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118
Method,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#KABAT,0.6667,0.6667,0.8889,0.8000,0.8889,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.8889,0.8889,0.6667,1.0000,0.8889,0.6667,0.6667,0.8889,0.8889,0.8000,0.6667,0.6667,0.6667,1.0000,0.0000,0.8889,0.0000,0.8889,1.0000,0.6667,1.0000,1.0000,1.0000,0.6667,0.6667,0.8889,0.0000,0.6667,0.6667,0.8889,0.0000,0.0000,0.6667,0.6667,0.8889,0.8889,0.6667,0.6667,0.8889,1.0000,1.0000,0.8000,1.0000,0.6667,0.8000,0.6667,0.0000,0.6667,0.9333,0.5333,0.5333,0.8000,0.0000,0.6667,0.8889,0.6667,0.0000,0.6667,0.6667,0.0000,0.6667,0.6667,0.8889,0.0000,1.0000,0.8889,0.8000,0.6667,0.8889,0.6667,1.0000,0.8889,0.0000,0.6667,0.8000,0.6667,0.8889,0.8000,0.8889,0.8889,1.0000,0.6667,0.0000,0.0000,0.8000,1.0000,0.6667,0.0000,0.6667,0.8889,0.8000,0.0000,0.6667,0.0000,0.0000,0.0000,0.6667,0.8889,0.6667,0.6667,0.8889,0.8000,0.8889,0.8889,1.0000,1.0000
#JORES,0.6857,0.6857,0.9143,0.9000,0.9143,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.9143,0.9143,0.6857,1.0000,0.9143,0.6857,0.6857,0.9143,0.9143,0.9000,0.6857,0.6857,0.6857,1.0000,0.0000,0.9143,0.0000,0.9143,1.0000,0.6857,1.0000,1.0000,1.0000,0.6857,0.6857,0.9143,0.0000,0.6857,0.6857,0.9143,0.0000,0.0000,0.6857,0.6857,0.9143,0.9143,0.6857,0.6857,0.9143,1.0000,1.0000,0.9000,1.0000,0.6857,0.9000,0.6857,0.0000,0.6857,0.8571,0.8571,0.8571,0.9000,0.0000,0.6857,0.9143,0.6857,0.0000,0.6857,0.6857,0.0000,0.6857,0.6857,0.9143,0.0000,1.0000,0.9143,0.9000,0.6857,0.9143,0.6857,1.0000,0.9143,0.0000,0.6857,0.9000,0.6857,0.9143,0.9000,0.9143,0.9143,1.0000,0.6857,0.0000,0.0000,0.9000,1.0000,0.6857,0.0000,0.6857,0.9143,0.9000,0.0000,0.6857,0.0000,0.0000,0.0000,0.6857,0.9143,0.6857,0.6857,0.9143,0.9000,0.9143,0.9143,1.0000,1.0000
#SCHNEIDER,0.2500,0.2500,0.5944,0.5000,0.5944,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.5944,0.5944,0.2500,1.0000,0.5944,0.2500,0.2500,0.5944,0.5944,0.5000,0.2500,0.2500,0.2500,1.0000,0.0000,0.5944,0.0000,0.5944,1.0000,0.2500,1.0000,1.0000,1.0000,0.2500,0.2500,0.5944,0.0000,0.2500,0.2500,0.5944,0.0000,0.0000,0.2500,0.2500,0.5944,0.5944,0.2500,0.2500,0.5944,1.0000,1.0000,0.5000,1.0000,0.2500,0.5000,0.2500,0.0000,0.2500,0.7500,0.5000,0.5000,0.5000,0.0000,0.2500,0.5944,0.2500,0.0000,0.2500,0.2500,0.0000,0.2500,0.2500,0.5944,0.0000,1.0000,0.5944,0.5000,0.2500,0.5944,0.2500,1.0000,0.5944,0.0000,0.2500,0.5000,0.2500,0.5944,0.5000,0.5944,0.5944,1.0000,0.2500,0.0000,0.0000,0.5000,1.0000,0.2500,0.0000,0.2500,0.5944,0.5000,0.0000,0.2500,0.0000,0.0000,0.0000,0.2500,0.5944,0.2500,0.2500,0.5944,0.5000,0.5944,0.5944,1.0000,1.0000
#SHENKIN,0.3905,0.3905,0.7484,0.6667,0.7484,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7484,0.7484,0.3905,1.0000,0.7484,0.3905,0.3905,0.7484,0.7484,0.6667,0.3905,0.3905,0.3905,1.0000,0.0000,0.7484,0.0000,0.7484,1.0000,0.3905,1.0000,1.0000,1.0000,0.3905,0.3905,0.7484,0.0000,0.3905,0.3905,0.7484,0.0000,0.0000,0.3905,0.3905,0.7484,0.7484,0.3905,0.3905,0.7484,1.0000,1.0000,0.6667,1.0000,0.3905,0.6667,0.3905,0.0000,0.3905,0.8619,0.6667,0.6667,0.6667,0.0000,0.3905,0.7484,0.3905,0.0000,0.3905,0.3905,0.0000,0.3905,0.3905,0.7484,0.0000,1.0000,0.7484,0.6667,0.3905,0.7484,0.3905,1.0000,0.7484,0.0000,0.3905,0.6667,0.3905,0.7484,0.6667,0.7484,0.7484,1.0000,0.3905,0.0000,0.0000,0.6667,1.0000,0.3905,0.0000,0.3905,0.7484,0.6667,0.0000,0.3905,0.0000,0.0000,0.0000,0.3905,0.7484,0.3905,0.3905,0.7484,0.6667,0.7484,0.7484,1.0000,1.0000
#GERSTEIN,0.2499,0.2499,0.5943,0.5000,0.5943,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.5943,0.5943,0.2499,1.0000,0.5943,0.249

In [307]:
methods = ["KABAT,JORES,SCHNEIDER,SHENKIN,GERSTEIN,TAYLOR_GAPS,TAYLOR_NO_GAPS,ZVELIBIL,KARLIN,ARMON,THOMPSON,NOT_LANCET,MIRNY,WILLIAMSON,LANDGRAF,SANDER,VALDAR"]
print(methods)                                                                                                                                                   
cons_method = input("Please select which scoring method you would like to use: ")                                                                                
scores = list(df6.loc["#{}".format(cons_method)])                                                                                                                
oc_set = set()                                                                                                                                                   
pos = []                                                                                                                                                         
res = []                                                                                                                                                         
for idx, val in enumerate(scores):                                                                                                                               
    if val <= 0.2:                                                                                                                                               
        if val not in oc_set:                                                                                                                                    
            oc_set.add(val)                                                                                                                                      
            pos.append(idx)                                                                                                                                      
        else:                                                                                                                                                    
            res.append(idx)                                                                                                                                      
variable_pos = res + pos                                                                                                                                         
variable_pos.sort()                                                                                                                                              
print("The predicted variable and non-conserved residues are: ", variable_pos)                                                                                   

['KABAT,JORES,SCHNEIDER,SHENKIN,GERSTEIN,TAYLOR_GAPS,TAYLOR_NO_GAPS,ZVELIBIL,KARLIN,ARMON,THOMPSON,NOT_LANCET,MIRNY,WILLIAMSON,LANDGRAF,SANDER,VALDAR']
Please select which scoring method you would like to use: KABAT
The predicted variable and non-conserved residues are:  [27, 29, 39, 43, 44, 59, 65, 69, 72, 76, 85, 95, 96, 100, 104, 106, 107, 108]


In [308]:
alignment = AlignIO.read("domain_seq_aligned.aln", "clustal")                          
l = []                                                                                 
for i in alignment.__dict__['_records']:                                               
    l.append([i.id, "".join(i.seq)])                                                   
df7 = pd.DataFrame(l)                                                                  
df7.columns = ['id', 'seq']                                                            
df8 = pd.DataFrame([list(word) for word in list(df7.seq)], index=df4["new_id"])                             
var_list = []                                                                          
for v in variable_pos:                                                                 
    var_list.append(list(df8.loc[:, v]))                                               
df9 = pd.DataFrame(var_list)                                                          
df9 = df9.T                                                                          
df9.index = df8.index                                                                 
actual_pos = []                                                                        
for v in variable_pos:                                                                 
    actual_pos.append(v)                                                               
df9.columns = actual_pos 

In [310]:
df8

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118
new_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
>P0A6P5_4-119,V,V,A,L,V,G,R,P,N,V,G,K,S,T,L,F,N,R,L,T,R,T,R,D,A,L,V,A,D,F,P,G,L,T,R,D,R,K,Y,G,R,A,E,I,E,G,R,E,F,I,C,I,D,T,G,G,I,D,G,T,E,-,-,-,D,G,V,E,T,R,M,A,E,Q,S,L,L,A,I,E,E,A,D,V,V,L,F,M,V,D,A,R,A,G,L,M,P,A,D,E,A,I,A,K,H,L,R,S,R,E,K,P,T,F,L,V,A,N,K
>P0A6P5_204-322,K,L,A,I,V,G,R,P,N,V,G,K,S,T,L,T,N,R,I,L,G,E,E,R,V,V,V,Y,D,M,P,G,T,T,R,D,S,I,Y,I,P,M,E,R,D,G,R,E,Y,V,L,I,D,T,A,G,V,R,K,R,G,K,I,T,D,A,V,E,K,F,S,V,I,K,T,L,Q,A,I,E,D,A,N,V,V,M,L,V,I,D,A,R,E,G,I,S,D,Q,D,L,S,L,L,G,F,I,L,N,S,G,R,S,L,V,I,V,V,N,K
>P50743_5-120,V,V,A,I,V,G,R,P,N,V,G,K,S,T,I,F,N,R,I,A,G,E,R,I,S,I,V,E,D,T,P,G,V,T,R,D,R,I,Y,S,S,A,E,W,L,N,Y,D,F,N,L,I,D,T,G,G,I,D,I,G,D,-,-,-,E,P,F,L,A,Q,I,R,Q,Q,A,E,I,A,M,D,E,A,D,V,I,I,F,M,V,N,G,R,E,G,V,T,A,A,D,E,E,V,A,K,I,L,Y,R,T,K,K,P,V,V,L,A,V,N,K
>P50743_177-295,Q,F,C,L,I,G,R,P,N,V,G,K,S,S,L,V,N,A,M,L,G,E,E,R,V,I,V,S,N,V,A,G,T,T,R,D,A,V,D,T,S,F,T,Y,N,Q,Q,E,F,V,I,V,D,T,A,G,M,R,K,K,G,K,V,Y,E,T,T,E,K,Y,S,V,L,R,A,L,K,A,I,D,R,S,E,V,V,A,V,V,L,D,G,E,E,G,I,I,E,Q,D,K,R,I,A,G,Y,A,H,E,A,G,K,A,V,V,I,V,V,N,K


In [311]:
df9

,27,29,39,43,44,59,65,69,72,76,85,95,96,100,104,106,107,108
new_id,,,,,,,,,,,,,,,,,,
>P0A6P5_4-119,A,F,G,I,E,T,G,R,E,L,L,M,P,A,H,R,S,R
>P0A6P5_204-322,Y,M,I,R,D,R,A,F,I,Q,M,S,D,S,F,L,N,S
>P50743_5-120,E,T,S,W,L,G,P,Q,Q,I,I,T,A,E,I,Y,R,T
>P50743_177-295,S,V,T,Y,N,K,T,Y,L,K,A,I,E,R,Y,H,E,A


In [313]:
df10 = []
for idx, row in df9.iterrows():
    offset = int(idx.split("_")[-1].split("-")[0])
    s = list(df8.loc[idx,:])
    s1 = list(enumerate(s))
    s2 = [i for i in s1 if i[1]!="-"]
    s3 = [(i[0],i[1],idx+offset) for idx,i in enumerate(s2)]
    idxs = list(df9.columns)
    df10.append([j[2] for i in idxs for j in s3 if j[0]==i])
df11 = pd.DataFrame(df10,columns= list(df9.columns))
df11['id'] = list(df9.index)
df11 = df11.set_index('id')
df11

,27,29,39,43,44,59,65,69,72,76,85,95,96,100,104,106,107,108
id,,,,,,,,,,,,,,,,,,
>P0A6P5_4-119,31,33,43,47,48,63,66,70,73,77,86,96,97,101,105,107,108,109
>P0A6P5_204-322,231,233,243,247,248,263,269,273,276,280,289,299,300,304,308,310,311,312
>P50743_5-120,32,34,44,48,49,64,67,71,74,78,87,97,98,102,106,108,109,110
>P50743_177-295,204,206,216,220,221,236,242,246,249,253,262,272,273,277,281,283,284,285


In [314]:
def waitUntilDownloadCompleted(maxTime=600):
    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get('chrome://downloads')
    endTime = time.time() + maxTime
    while True:
        try:
            downloadPercentage = driver.execute_script(
                "return document.querySelector('downloads-manager').shadowRoot.querySelector('#downloadsList downloads-item').shadowRoot.querySelector('#progress').value")
            if downloadPercentage == 100:
                print("File downloaded")
                return downloadPercentage
        except:
            pass
        time.sleep(1)
        if time.time() > endTime:
            break

In [315]:
def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [316]:
msa_path = str(os.getcwd()+r'/domain_seq_aligned.aln')
print(msa_path)

/home/dell/Desktop/CPDAS/CPDA-ConservedProteinDomainAnalysis/venv/domain_seq_aligned.aln


In [318]:
PATH = "/home/dell/WebDrivers/chromedriver"
chrome_options = webdriver.ChromeOptions()
preferences = {
                "profile.default_content_settings.popups": 0,
                "download.default_directory": os.getcwd()+'/pdb-files',
                "directory_upgrade": True
            }
chrome_options.add_experimental_option('prefs', preferences)
chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")
listofseq=[]
pdb_paths= []
for record in SeqIO.parse("domain_seq.fasta", "fasta"):
    listofseq.append(record.id)
seq_ids = set()
for seq_id in listofseq:
    seq_id_1 = seq_id[3:9]
    if seq_id_1 not in seq_ids:
        seq_ids.add(seq_id_1)
        s = Service(PATH)
        driver=webdriver.Chrome(service=s, options=chrome_options)
        driver.implicitly_wait(75)
        driver.get("https://alphafold.ebi.ac.uk/")
        driver.implicitly_wait(45)
        query_input = driver.find_element(by=By.XPATH, value="/html/body/div[1]/app-root/app-header/div/div[2]/div/div/div[2]/app-header-search/div[1]/input").send_keys(seq_id_1)
        search=driver.find_element(by=By.XPATH, value="/html/body/div[1]/app-root/app-header/div/div[2]/div/div/div[2]/app-header-search/button/span").click()
        select_result= driver.find_element(by=By.CLASS_NAME, value="vf-link").click()
        download_pdb=driver.find_element(by=By.LINK_TEXT, value="PDB file").click()
        waitUntilDownloadCompleted(75)
        PDB_Path = newest(os.getcwd()+'/pdb-files')
        os.rename(PDB_Path, os.getcwd()+'/pdb-files/{}.pdb'.format(seq_id_1))
        PDB_Path1 = newest(os.getcwd()+r'/pdb-files')
        PDB_name = os.path.basename(PDB_Path1)
        print(PDB_name)
        seq_index = seq_id[20:]
        seq_index_list = seq_index.split("-")
        seq_index_start = int(seq_index_list[0])
        seq_index_stop = int(seq_index_list[1])
        os.chdir(os.getcwd()+'/pdb-files/')
        new_name = str("{}_{}-{}.pdb".format(seq_id_1, seq_index_start, seq_index_stop))
        seleres = "python pdb_selres.py -{}:{} {} > {}".format(seq_index_start, seq_index_stop, PDB_name, new_name)
        subprocess.call(seleres, shell=True)
        pdb_paths.append(newest(os.getcwd()))
        print("done")
        os.chdir('..')
    else:
        PDB_name = "{}.pdb".format(seq_id_1)
        seq_index = seq_id[20:]
        seq_index_list = seq_index.split("-")
        seq_index_start = int(seq_index_list[0])
        seq_index_stop = int(seq_index_list[1])
        os.chdir(os.getcwd()+'/pdb-files/')
        new_name = str("{}_{}_{}.pdb".format(seq_id_1, seq_index_start, seq_index_stop))
        seleres = "python pdb_selres.py -{}:{} {}>{}".format(seq_index_start, seq_index_stop, PDB_name, new_name)
        subprocess.call(seleres, shell=True)
        pdb_paths.append(newest(os.getcwd()))
        print("done")
        os.chdir('..')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".vf-link"}
  (Session info: headless chrome=100.0.4896.127)
Stacktrace:
#0 0x5614a8dd87d3 <unknown>
#1 0x5614a8b34688 <unknown>
#2 0x5614a8b6ac21 <unknown>
#3 0x5614a8b6ade1 <unknown>
#4 0x5614a8b9dd74 <unknown>
#5 0x5614a8b886dd <unknown>
#6 0x5614a8b9ba0c <unknown>
#7 0x5614a8b885a3 <unknown>
#8 0x5614a8b5dddc <unknown>
#9 0x5614a8b5ede5 <unknown>
#10 0x5614a8e0949d <unknown>
#11 0x5614a8e2260c <unknown>
#12 0x5614a8e0b205 <unknown>
#13 0x5614a8e22ee5 <unknown>
#14 0x5614a8dff070 <unknown>
#15 0x5614a8e3e488 <unknown>
#16 0x5614a8e3e60c <unknown>
#17 0x5614a8e57c6d <unknown>
#18 0x7f75deb99609 <unknown>


In [191]:
print(pdb_paths)

['/home/dell/Desktop/CPDAS/venv/pdb-files/P0A6P5_4_119.pdb', '/home/dell/Desktop/CPDAS/venv/pdb-files/P0A6P5_204_322.pdb', '/home/dell/Desktop/CPDAS/venv/pdb-files/P50743_5_120.pdb', '/home/dell/Desktop/CPDAS/venv/pdb-files/P50743_177_295.pdb']


In [ ]:
alignment = AlignIO.read("domain_seq_aligned.aln", "clustal")                                                                                                                                                                                                                                  
ids = []                                                                                                                                                                                                                                                                                       
for a in alignment.__dict__['_records']:                                                                                                                                                                                                                                                       
    ids.append(a.id)                                                                                                                                                                                                                                                                           
id_path_dict = dict(zip(ids, pdb_paths))                                                                                                                                                                                                                                                       
PATH = "/home/dell/WebDrivers/chromedriver"                                                                                                                                                                                                                                                    
chrome_options = webdriver.ChromeOptions()                                                                                                                                                                                                                                                     
preferences = {                                                                                                                                                                                                                                                                                
    "profile.default_content_settings.popups": 0,                                                                                                                                                                                                                                              
    "download.default_directory": os.getcwd() + os.path.sep,                                                                                                                                                                                                                                   
    "directory_upgrade": True                                                                                                                                                                                                                                                                  
}                                                                                                                                                                                                                                                                                              
s = Service(PATH)                                                                                                                                                                                                                                                                              
driver = webdriver.Chrome(service=s, options=chrome_options)                                                                                                                                                                                                                                   
msa_path = str(os.getcwd()+r'/domain_seq_aligned.aln')                                                                                                                                                                                                                                         
for key, value in id_path_dict.items():                                                                                                                                                                                                                                                        
        seq_id = key                                                                                                                                                                                                                                                                           
        path = value                                                                                                                                                                                                                                                                           
        driver.implicitly_wait(75)                                                                                                                                                                                                                                                             
        driver.get("https://consurf.tau.ac.il/")                                                                                                                                                                                                                                               
        aa_button = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div/form/div[2]/label/input").click()                                                                                                                                                                            
        adv_options = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/form/table/tbody/tr/td[2]/a').click()                                                                                                                                                                      
        yes_structure = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/form/div[1]/div[1]/label/input').click()                                                                                                                                                                 
        upload_structure = driver.find_element(by=By.NAME, value='pdb_FILE')                                                                                                                                                                                                                   
        upload_structure.send_keys(path)                                                                                                                                                                                                                                                       
        chain_identifier = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/form/div[1]/div[2]/input[4]').send_keys('A')                                                                                                                                                          
        MSA_yes = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/form/div[3]/div[1]/div[1]/label/input').click()                                                                                                                                                                
        upload_msa = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/form/div[3]/div[2]/input[1]').send_keys(msa_path)                                                                                                                                                           
        try:                                                                                                                                                                                                                                                                                   
            ddmenu = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div/form/div[3]/div[2]/div[2]/select")                                                                                                                                                                          
            ddmenu = Select(ddmenu)                                                                                                                                                                                                                                                            
            ddmenu.select_by_visible_text(str(seq_id))                                                                                                                                                                                                                                         
        except NoSuchElementException:                                                                                                                                                                                                                                                         
            seq_text = driver.find_element(by=By.NAME, value="msa_SEQNAME")                                                                                                                                                                                                                    
            seq_text.send_keys(str(seq_id))                                                                                                                                                                                                                                                    
        try:                                                                                                                                                                                                                                                                                   
            update_sel = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div/form/div[3]/div[2]/div[2]/a[2]").click()                                                                                                                                                                
        except NoSuchElementException:                                                                                                                                                                                                                                                         
            link_text = driver.find_element(by=By.LINK_TEXT, value="update selection").click()                                                                                                                                                                                                 
        try:                                                                                                                                                                                                                                                                                   
            no_email = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div/form/div[13]/label/input").click()                                                                                                                                                                        
        except NoSuchElementException:                                                                                                                                                                                                                                                         
            no_mail_check = driver.find_element(by=By.NAME, value="send_user_mail").click()                                                                                                                                                                                                    
        try:                                                                                                                                                                                                                                                                                   
            submit_button = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div/form/input[3]").click()                                                                                                                                                                              
        except NoSuchElementException:                                                                                                                                                                                                                                                         
            driver.find_element(by=By.NAME, value="submitForm").click()                                                                                                                                                                                                                        
        try:                                                                                                                                                                                                                                                                                   
            alert = driver.switch_to.alert                                                                                                                                                                                                                                                     
            alert_msg = driver.switch_to.alert.text                                                                                                                                                                                                                                            
            print(alert_msg)                                                                                                                                                                                                                                                                   
            alert.accept()                                                                                                                                                                                                                                                                     
        except NoAlertPresentException:                                                                                                                                                                                                                                                        
            print("exception handled")                                                                                                                                                                                                                                                         
        WebDriverWait(driver, 175).until(EC.presence_of_element_located((By.CLASS_NAME, "output_title")))                                                                                                                                                                                      
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "click!"))).click()                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [215]:
import requests
import os
from bs4 import BeautifulSoup
import csv

PATH = "/home/dell/WebDrivers/chromedriver"
newpath = (os.getcwd() + '/getarea_results')
if not os.path.exists(newpath):
    os.makedirs(newpath)
chrome_options = webdriver.ChromeOptions()
preferences = {
    "profile.default_content_settings.popups": 0,
    "download.default_directory": (os.getcwd() + '/getarea_results'),
    "directory_upgrade": True
}
chrome_options.add_experimental_option('prefs', preferences)
chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")
s = Service(PATH)
driver = webdriver.Chrome(service=s, options=chrome_options)
for path in pdb_paths:
    email = "rajvisrivastav@ggmail.com" #input('Please enter your email ID')
    driver.implicitly_wait(20)
    driver.get("http://curie.utmb.edu/getarea.html")
    pdb_upload = driver.find_element(by=By.XPATH, value="/html/body/center/form/table/tbody/tr[1]/td/div/large/input[1]").send_keys(path)
    email = driver.find_element(by=By.XPATH, value='/html/body/center/form/table/tbody/tr[1]/td/div/large/input[4]').send_keys(email)
    submit = driver.find_element(by=By.XPATH, value='/html/body/center/form/table/tbody/tr[1]/td/div/large/input[5]').click()
    WebDriverWait(driver, 175).until(EC.presence_of_element_located((By.CSS_SELECTOR, "head > title")))
    soup = BeautifulSoup(driver.page_source, "lxml")
    prelist = soup.find_all('pre')
    info = []
    for pre in prelist:
        info.append(pre.text)
    del info[0:4]
    info[0] = "Residue   Res_No   Total   Apolar  Backbone Sidechain Ratio(%) In/Out"
    file_name = (os.path.basename(path)).split('.')
    os.chdir(os.getcwd()+'/getarea_results')
    with open ('getarea.txt', 'w') as file:
            for i in info:
                file.write('\n'+ i)
    with open ('getarea.txt') as fin, open( file_name[0]+'.csv', 'w') as fout:
        o=csv.writer(fout)
        for line in fin:
            o.writerow(line.split())
    os.chdir('..')
print('done')

done


In [283]:
print(listofseq)

['sp|P0A6P5|DER_ECOLI|4-119', 'sp|P0A6P5|DER_ECOLI|204-322', 'sp|P50743|DER_BACSU|5-120', 'sp|P50743|DER_BACSU|177-295']


In [278]:
path = os.getcwd()+'/getarea_results'
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

path1 = str(all_files[0])
getarea_df = pd.read_csv(path1)
csv_file = (os.path.basename(all_files[0])).split(".")
name = csv_file[0]

print(name)
getarea_df

P0A6P5_4_119


,Residue,Res_No,Total,Apolar,Backbone,Sidechain,Ratio(%),In/Out
0,ALA,6,0.00,0.00,0.00,0.00,0.0,i
1,LEU,7,0.00,0.00,0.00,0.00,0.0,i
2,VAL,8,2.11,2.11,0.00,2.11,1.7,i
3,GLY,9,7.04,0.00,7.04,0.00,8.1,i
4,ARG,10,44.62,8.55,1.41,43.20,22.1,NaN
5,PRO,11,95.19,88.16,13.64,81.54,77.5,o
6,ASN,12,130.07,44.22,21.03,109.03,95.4,o
7,VAL,13,0.72,0.68,0.25,0.48,0.4,i
8,GLY,14,15.98,15.70,15.98,0.00,18.3,i
9,LYS,15,15.61,8.70,1.35,14.25,8.7,i
